In [2]:
from bs4 import BeautifulSoup  
import pandas as pd
from tqdm import tqdm_notebook
import nltk
import re
from urllib.request import urlopen

In [1]:
#크롤링코드작성

In [4]:
!pip install konlpy
!pip install mat

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 8.2MB/s 
     |████████████████████████████████| 460kB 39.3MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


     |████████████████████████████████| 18.1MB 1.6MB/s 
     |████████████████████████████████| 645kB 43.4MB/s 
  Created wheel for mat: filename=mat-1.0.2-cp37-none-any.whl size=11700 sha256=233a055c42dea4fa0fb65b59ffab71929763d01a7dc019f5167fc81e11895edd
  Stored in directory: /root/.cache/pip/wheels/36/c5/83/552de5e585311c22c525dee7e3c2ae10e12ad6d0d9570dfb02
Successfully built mat
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import linear_kernel
from konlpy.tag import Mecab
import pandas as pd
import tensorflow as tf
import os
import getpass
import numpy as np
from numpy import dot
from numpy.linalg import norm
import mat 

#dataframe에 null값이 있는 경우 공백을 넣어 null값 제거 
def avoid_null(data, header):
  data[header] = data[header].fillna('') 
  return data[header]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_csv('/content/drive/MyDrive/data/financial_word.CSV', encoding='cp949')
df

,id,word,text
0,1,우대금리,"우대금리란 그야 말로 하늘이 갈라지고, 천지가 지랄을 해도 절대로 돈을 띵가묵지 않..."
1,2,경제,경제는 경세제민을 줄인 말입니다. 세상을 경영해 백성을 부유하게 한다는 뜻이죠. 쉽...
2,3,경기,총체적인 경제의 활동기운
3,4,수출경기,수출과 관련된 경제의 활동기운'을 줄인말
4,5,내수경기,?'국내수요와 관련된 경제의 활동기운'을 줄인말.
...,...,...,...
63,64,선물,미리 계약하는 것. 예)2200년에 석유 한방울을 1달러에 사고 팔기로 해요.
64,65,선물환,미리 외국돈을 사고팔기로 계약하는 것. 예)2200년에 1달러를 100원에 사고 팔...
65,66,옵션,마음대로 거래할수 있는 권리
66,67,콜옵션,"사고 싶으면 사고, 사기 싫으면 안 살거야 살수있는 권리"


In [7]:
import re

#한글, 숫자, 영어 빼고 전부 제거 (정규화처리)
def sub_special(s):
  return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',s)

In [8]:
df.loc[:,"text"] = df.loc[:,"text"].apply(sub_special)

In [9]:
df #확인 완료 

,id,word,text
0,1,우대금리,우대금리란 그야 말로 하늘이 갈라지고 천지가 지랄을 해도 절대로 돈을 띵가묵지 않을...
1,2,경제,경제는 경세제민을 줄인 말입니다 세상을 경영해 백성을 부유하게 한다는 뜻이죠 쉽게 ...
2,3,경기,총체적인 경제의 활동기운
3,4,수출경기,수출과 관련된 경제의 활동기운을 줄인말
4,5,내수경기,국내수요와 관련된 경제의 활동기운을 줄인말
...,...,...,...
63,64,선물,미리 계약하는 것 예2200년에 석유 한방울을 1달러에 사고 팔기로 해요
64,65,선물환,미리 외국돈을 사고팔기로 계약하는 것 예2200년에 1달러를 100원에 사고 팔기로 해요
65,66,옵션,마음대로 거래할수 있는 권리
66,67,콜옵션,사고 싶으면 사고 사기 싫으면 안 살거야 살수있는 권리


In [10]:
duplicate = df[df.duplicated(['word'])]
df = df.drop_duplicates(['word'], keep = 'first')

#중복자료제거 

In [11]:
# 전체를 합치기
df['all']=df['word'] + df['text']
df['all'][0]

'우대금리우대금리란 그야 말로 하늘이 갈라지고 천지가 지랄을 해도 절대로 돈을 띵가묵지 않을 기업에게 빌려줄 때 적용하는 금리입니다'

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
df['all'] = df['all'].fillna('')

In [14]:
tfidf_matrix = tfidf.fit_transform(df['all'])
# overview에 대해서 tf-idf 수행
print(tfidf_matrix.shape)

(68, 304)


In [15]:
tfidf_matrix

<68x304 sparse matrix of type '<class 'numpy.float64'>'
	with 397 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [18]:
indices = pd.Series(df.index, index=df['word']).drop_duplicates()
print(indices.head())

word
우대금리    0
경제      1
경기      2
수출경기    3
내수경기    4
dtype: int64


In [25]:
def get_recomm(title, cosine_sim=cosine_sim):
    choice = []
    idx = indices[title]

    # 코사인 유사도
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 5개의 단어 조회
    sim_scores = sim_scores[1:6] 
    # 0부터 하면 본인 (단어)이 나와버림 ;; 

    # 가장 유사한 5개의 영화 인덱스 조회 
    word_indices = [i[0] for i in sim_scores]
    
    for i in range(5):
        choice.append(df['word'][word_indices[i]])
        #인덱스로 리턴해서 추가 
  
    # 가장 유사한 5개의 단어 출력
    print('***단어 추천 순위***')
    for i in range(5):
        print(str(i+1) + '순위 : ' + choice[i])

In [26]:
get_recomm('옵션')

***단어 추천 순위***
1순위 : 풋옵션
2순위 : 통화
3순위 : 콜옵션
4순위 : 유가증권
5순위 : 외환


In [23]:
print(get_recomm('우대금리'))

***단어 추천 순위***
1순위 : 우대금리
2순위 : 국고채금리
3순위 : 외환시장
4순위 : 기준금리
5순위 : 여신금리
None
